# Background

* ML solutions that predict ship moves may not be optimal at predicting moments to spawn.
* In previous Halite competitions, many participants ran an algorithmic spawn schedule alonside a ML ship mover.
* This notebook is an attempt to predict or imitate spawn event generation through ML. Such a component could be used with a separate ML ship mover.
* We'll use real games from a top player, and Google's TabNet, to automate ship spawning.

# Imports

In [ ]:
# In the live halite competition, internet is off. You need to include used libraries as text.
!pip install pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier


In [ ]:
import torch
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from random import shuffle
np.random.seed(0)
import os
from matplotlib import pyplot as plt
%matplotlib inline

# Data Preparation

* The dataset here is 1550 games played by the top 8 submissions of 1 top team. 
* The dataset has already been stripped of steps where we don't have 500 halite, or any bases.
* The first 10 steps of each game aren't used. Almost all strategies start by creating 10 ships.
* You can build your own dataset using the Halite Scraper API. 


In [ ]:
train = pd.read_csv('../input/halitespawn/spawn.csv', header=0)
print(train.columns)

In [ ]:
# create some more features
train['step<20'] = (train['step']<20).astype(int)
train['step<100'] = (train['step']<100).astype(int)
train['step_100_200'] = ((train['step']>=100) & train['step']<200).astype(int)
train['step_200_300'] = ((train['step']>=200) & train['step']<300).astype(int)
train['step_300_400'] = (train['step']>=300).astype(int)
train['halite_enough1000'] = (train['halite_m']>1000).astype(int)
train['ships<10'] = (train['ships_m']<=10).astype(int)
train['ships<20'] = (train['ships_m']<=20).astype(int)
train['ships<30'] = (train['ships_m']<=30).astype(int)
train['ships<40'] = (train['ships_m']<=40).astype(int)
train['halite_most'] = (train['halite_m'] > train['halite_max']).astype(int)
train['halite_more'] = (train['halite_m'] > (train['halite_y']/3)).astype(int)
train['shipswithcargo_m'] = (train['ships_m']-train['shipsg0_m']).astype(int)
train['shipswithcargo_y'] = (train['ships_y']-train['shipsg0_y']).astype(int)
train['ships_less_avg'] = (train['ships_m'] < (train['ships_y']/3)).astype(int)
train['ships_less_min'] = (train['ships_m'] < train['ships_min']).astype(int)
train['avail_per_ship_m'] = (train['halite_total_now']/(train['ships_m']+.010)).astype(int)
train['avail_per_ship_all'] = (train['halite_total_now']/(train['ships_m']+train['ships_y']+.001)).astype(int)

We'll use features like the number of our ships, our cargo, our halite, as well as the same of opponents.

In [ ]:
target = "spawn"

int_columns = [
    # _m==mine, _y==yours
    'step', 'halite_total_start', 'halite_total_now', 'ships_m', 'ships_y', 'ships_min', 'ships_max', \
    'bases_y', 'halite_m', 'halite_y', 'halite_max', 'cargo_m', 'cargo_y', 'shipsg0_m', 'shipsg0_y', 'ship_lost', \
    'step<20', 'step<100', 'step_100_200', 'step_200_300', 'step_300_400', \
    'halite_enough1000', 'ships<10', 'ships<20', 'ships<30', 'ships<40', \
    'halite_most', 'halite_more', 'shipswithcargo_m', 'shipswithcargo_y', 'ships_less_avg', 'ships_less_min', \
    'avail_per_ship_m', 'avail_per_ship_all', \
]

feature_columns = ( int_columns + [target])


In [ ]:
matches = train.match_id.unique()
shuffle(matches)
print(f'Dataset is {len(train)} steps from {len(matches)} matches')

train_matches = matches[:80*len(matches)//100]
test_matches = matches[-10:]
valid_matches = matches[len(train_matches):-10]
print(f'train/valid/test matches: {len(train_matches)}/{len(valid_matches)}/{len(test_matches)}')

train_indices = train.index[train['match_id'].isin(train_matches)].to_list()
valid_indices = train.index[train['match_id'].isin(valid_matches)].to_list()
test_indices = train.index[train['match_id'].isin(test_matches)].to_list()

unused_feat = ['match_id']

features = [ col for col in train.columns if col not in unused_feat+[target]] 


# Network Parameters

In [ ]:
clf = TabNetClassifier( # default tabnet hyperparameters
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    lambda_sparse=1e-4, momentum=0.3, clip_value=2.,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95, "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15
)

# Training

In [ ]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

In [ ]:
max_epochs = 90

clf.fit(
    X_train=X_train, y_train=y_train,
    X_valid=X_valid, y_valid=y_valid,
    max_epochs=max_epochs, patience=50,
    batch_size=16384, virtual_batch_size=256
)

# Results

We can see what the most important features are. Some look a little surprising.

In [ ]:
[feature_columns[i] for i in np.argsort(clf.feature_importances_)[::-1][:10]]

Here, feature importance masks that indicate which features are selected at each step.

In [ ]:
explain_matrix, masks = clf.explain(X_test)
fig, axs = plt.subplots(1, 5, figsize=(20,20))

for i in range(5):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")

# Loss and Accuracy graphs

In [ ]:
plt.plot(clf.history['train']['loss'][6:])
plt.plot(clf.history['valid']['loss'][6:])

In [ ]:
plt.plot([-x for x in clf.history['train']['metric']][6:])
plt.plot([-x for x in clf.history['valid']['metric']][6:])


# Predictions

In [ ]:
y_pred = clf.predict(X_test)
test_acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f"FINAL TEST SCORE : {test_acc}")

We can try to visualise performance.
These event plots are 400 steps long from left to right.

In [ ]:
episodes = []
test_matches = []
match_id = 0
for n, (index, row) in enumerate(train.iloc[test_indices].iterrows()):
    if row['match_id'] != match_id:
        test_matches.append(row['match_id'])
        t = [i for i in range(10)]
        p = [i for i in range(10)]
        if match_id>0: episodes.append([t,p])
        match_id = row['match_id']
    if y_test[n]: t.append(row['step'])
    if y_pred[n]: p.append(row['step'])
episodes.append([t,p])



In [ ]:
colors = np.array([[1, 0, 0], [0, 0, 1]])
                   
for i in range(10):
    fig, axs = plt.subplots(1,1)
    plt.hlines(.5,0,380)  # Draw a horizontal line
    plt.eventplot(episodes[i], orientation='horizontal', colors=colors)
    axs.legend([f'Prediction for match {test_matches[i]}','Truth'], bbox_to_anchor=(0., 1.0, 1., .10), loc=3,ncol=2, mode="expand", borderaxespad=0.)
    plt.axis('off')
    plt.show()

# Conclusion

What may help accuracy? More features, more data.

Some spawn algorithms will use more complicated versions of how much halite remains for the number of ships on the board and the regen rate. This could be turned into a feature.

Other strategies will replace lost ships. This could also be featurised.

Is it worth it versus a simple rule for spawning? 